In [1]:
import torch
import gc
import logging

# del session.logger
# del session.epoch_logger
# del session
# gc.collect()
# torch.cuda.empty_cache()

import multiprocessing
from train.training_session_v2 import TrainingSessionV2, load_training_session
from config import TrainingConfigV2
from config import SimpleConvConfig

data_partition = {
    "gwilliams2023": {
        "testing_subjects": [19, 20, 21],
        "testing_tasks": [0],
    },
    "armeini2022": {
        "testing_subjects": [],
        "testing_tasks": [0, 1],
    },
}

model_config = SimpleConvConfig(
    # Str to list of possible conditions
    mel_normalization=False,
    conditions={
        "study": [],
        "subject": [],
    },
    # Channels
    in_channels=208,
    out_channels=80,
    hidden_dim=256,
    dropout=0.2,
    initial_batch_norm=True,
    # # Sensor layout settings
    # layout_dim=2,
    # layout_proj=True,
    # layout_scaling="minmax",
    # # Merger with spatial attn
    # merger=False,
    # merger_emb_type=None,
    # merger_emb_dim=0,
    # merger_channels=0,
    # merger_dropout=0.0,  # Float
    # merger_conditional=None,
    # Inital
    initial_linear=256,
    initial_depth=1,
    # Conditional layers
    conditional_layers=False,
    conditional_layers_dim=None,  # input or hidden_dim
    # Conv layer overall structure
    depth=4,
    kernel_size=3,
    growth=1.0,
    dilation_growth=2,
    dilation_period=5,
    glu=1,
    conv_dropout=0.2,
    dropout_input=0.1,
    batch_norm=True,
    half=True,
    cnn_pos_encoding=False,
    # Quantizer
    quantizer=False,
    num_codebooks=0,
    codebook_size=0,
    quantizer_commitment=0,
    quantizer_temp_init=0,
    quantizer_temp_min=0,
    quantizer_temp_decay=0,
    # Transformers Encoders
    transformer_input="continuous",
    transformer_encoder_emb="sinusoidal",
    transformer_encoder_layers=4,
    transformer_encoder_heads=4,
    # Conformer encoder variant
    rnn_type="conformer",
    depthwise_conv_kernel_size=15,
    use_group_norm=False,
    convolution_first=False,
    # Transformer Decoders
    transformer_decoder_emb=None,
    transformer_decoder_layers=0,
    transformer_decoder_heads=0,
    transformer_decoder_dim=0,
)

config = TrainingConfigV2(
    brain_encoder_config=model_config,
    data_partition=data_partition,
    # Ada lora settings
    # Around 100k total batches an epoch for gwilliams
    use_adalora=True,
    adalora_init_r=12,
    adalora_target_r=4,
    adalora_tinit=(654 * 3),  # 5% total steps
    adalora_tfinal=(654 * 8),  # 50-80% total steps
    adalora_deltaT=(654 * 1),  # 1-5% total steps
    adalora_lora_alpha=32,
    adalora_lora_dropout=0.1,
    adalora_total_step=(654 * 50),
    # Pre-processing parameters
    # Brain
    new_freq=200,
    frequency_bands={"all": (0.5, 80)},
    max_random_shift=1.0,
    window_size=4,
    window_stride=1,
    brain_clipping=None,
    baseline_window=0.5,
    notch_filter=True,
    scaling="both",
    delay=0.15,
    # Audio
    audio_model="openai/whisper-tiny.en",
    # Hyperparameters
    learning_rate=1e-4,
    weight_decay=1e-4,
    epochs=50,
    steps_per_epoch=450, # 654,
    batch_size=128,
    random_test_size=10,
    seed=42,
    mel_alignment_objectives = {
        "clip_loss": 0.6,
        "mse_loss": 0.4
    },
    latent_alignment_objectives= {
        "cosine_similarity": 0.4,
        "mse_loss": 0.4,
        "clip_loss": 0.6,
        "mmd_loss": 0.0
    },
    decode_timestamps=True,
)

config.brain_encoder_config.mel_normalization = False
config.learning_rate = 3e-4
config.batch_size = 256
config.steps_per_epoch = 654 # 654
config.epochs = 50

config.brain_encoder_config.in_channels = 269
config.decode_timestamps = True

# Sensor layout settings
config.brain_encoder_config.layout_dim=2
config.brain_encoder_config.layout_proj=False
config.brain_encoder_config.layout_scaling="midpoint"
# Merger with spatial attn
config.brain_encoder_config.merger=True
config.brain_encoder_config.merger_emb_type='mlp'
config.brain_encoder_config.merger_emb_dim=256
config.brain_encoder_config.merger_channels=269
config.brain_encoder_config.merger_dropout=0.1  # Float
config.brain_encoder_config.merger_conditional=None

# config.brain_clipping = 20

# config.brain_encoder_config.hidden_dim = 1024
# config.brain_encoder_config.initial_linear = 1024

session = TrainingSessionV2(
    config=config,
    studies={study: "audiotext" for study in data_partition.keys()},
    data_path="data",
    save_path="saves/phase3/combining/channel_merger_269_mlp_256_midpoint_2d",
    clear_cache=False,
    cache_name="/home/ubuntu/cache",
    download_studies=True,
)


# session = load_training_session(
#     save_path="saves/phase3/objectives/baseline_gwilliams_latent_loss_no_latent_alignment/epoch_39",
#     studies={"gwilliams2023": "audiotext"},
#     data_path="data",
#     cache_name="/home/ubuntu/cache",
# )

try:
    session.train(
        device="cuda",
        buffer_size=30,
        num_workers=(multiprocessing.cpu_count() - 2),
        max_cache_size=800,
        current_epoch=0,
    )
except KeyboardInterrupt as e:
    print("Exited")

# try:
#     session.pre_process_all_recordings(
#         buffer_size=30, num_workers=multiprocessing.cpu_count() - 10, max_cache_size=800
#     )
# except KeyboardInterrupt as e:
#     print("Exited")

Loading Gwilliams2023 with batch type audiotext
Loading Armeini2022 with batch type audiotext
Data partitioned on studies ['gwilliams2023', 'armeini2022'].
Train: 159, Unseen Task: 51, Unseen Subject: 12, Unseen Both: 4.

RNNEncoder initialized as conformer with 4 layers, 256 d_model, 4 nhead
	Embedding: sinusoidal, params: 6075392
SimpleConv initialized with 9079658 parameters, cond: ['study', 'subject']
Merger True, merger channels 269
ConvBlocks: 4, hidden_dim: 256, params 2626048
Using torch.bfloat16
Found 64 target modules for AdaLora: ['model.encoder.layers.0.self_attn.k_proj', 'model.encoder.layers.0.self_attn.v_proj', 'model.encoder.layers.0.self_attn.q_proj', 'model.encoder.layers.0.self_attn.out_proj', 'model.encoder.layers.0.fc1', 'model.encoder.layers.0.fc2', 'model.encoder.layers.1.self_attn.k_proj', 'model.encoder.layers.1.self_attn.v_proj', 'model.encoder.layers.1.self_attn.q_proj', 'model.encoder.layers.1.self_attn.out_proj', 'model.encoder.layers.1.fc1', 'model.encoder

2025-03-21 15:56:10,297	INFO worker.py:1841 -- Started a local Ray instance.
Training Epoch 1: 100%|██████████| 159/159 [18:37<00:00,  7.03s/it]


Testing done in 4.54m.
Epoch 1 done in 23.21m. 0.15m/recording.


New best epoch 1 with CER 0.9581 and BLEU 0.0047.
Mel Loss: 6.2077, Clip Loss: 9.5250, MSE: 1.2316
Mel accuracy: 0.0042, Top 5: 0.0226, Top 10: 0.0452


(raylet) Spilled 26412 MiB, 27 objects, write throughput 780 MiB/s. Set RAY_verbose_spill_logs=0 to disable this message.
Training Epoch 2: 100%|██████████| 159/159 [18:29<00:00,  6.98s/it]


Testing done in 3.82m.
Epoch 2 done in 22.32m. 0.14m/recording.


New best epoch 2 with CER 0.9536 and BLEU 0.0015.
Mel Loss: 5.8794, Clip Loss: 9.5309, MSE: 0.4023
Mel accuracy: 0.0046, Top 5: 0.0230, Top 10: 0.0458


...

Testing done in 4.56m.
Epoch 48 done in 23.34m. 0.15m/recording.


Training Epoch 49: 100%|██████████| 159/159 [18:42<00:00,  7.06s/it]


Testing done in 4.56m.
Epoch 49 done in 23.27m. 0.15m/recording.


Training Epoch 50: 100%|██████████| 159/159 [18:39<00:00,  7.04s/it]


Testing done in 4.69m.
Epoch 50 done in 23.35m. 0.15m/recording.


Training completed. Highest epoch at 5.


Test unseen_subject at epoch 5. Mel Loss: 5.7840, Clip Loss: 9.4633, MSE: 0.2650
Mel accuracy: 0.0040, Top 5: 0.0230, Top 10: 0.0458
BLEU: 0.0065, ROUGE-1: 0.0670, BERT: 0.3491, CER: 0.8161, SELF-BLEU: 0.4986


Test unseen_task at epoch 5. Mel Loss: 5.7719, Clip Loss: 9.4537, MSE: 0.2492
Mel accuracy: 0.0053, Top 5: 0.0249, Top 10: 0.0494
BLEU: 0.0060, ROUGE-1: 0.0558, BERT: 0.3662, CER: 0.8124, SELF-BLEU: 0.4274


Test unseen_both at epoch 5. Mel Loss: 5.7670, Clip Loss: 9.4447, MSE: 0.2505
Mel accuracy: 0.0037, Top 5: 0.0209, Top 10: 0.0468
BLEU: 0.0122, ROUGE-1: 0.1123, BERT: 0.3879, CER: 0.8247, SELF-BLEU: 0.5623


In [2]:
import torch
import gc
import logging

# del session.logger
# del session.epoch_logger
# del session
# gc.collect()
# torch.cuda.empty_cache()

import multiprocessing
from train.training_session_v2 import TrainingSessionV2, load_training_session
from config import TrainingConfigV2
from config import SimpleConvConfig

data_partition = {
    "gwilliams2023": {
        "testing_subjects": [19, 20, 21],
        "testing_tasks": [0],
    },
    "armeini2022": {
        "testing_subjects": [],
        "testing_tasks": [0, 1],
    },
}

model_config = SimpleConvConfig(
    # Str to list of possible conditions
    mel_normalization=False,
    conditions={
        "study": [],
        "subject": [],
    },
    # Channels
    in_channels=208,
    out_channels=80,
    hidden_dim=256,
    dropout=0.2,
    initial_batch_norm=True,
    # # Sensor layout settings
    # layout_dim=2,
    # layout_proj=True,
    # layout_scaling="minmax",
    # # Merger with spatial attn
    # merger=False,
    # merger_emb_type=None,
    # merger_emb_dim=0,
    # merger_channels=0,
    # merger_dropout=0.0,  # Float
    # merger_conditional=None,
    # Inital
    initial_linear=256,
    initial_depth=1,
    # Conditional layers
    conditional_layers=False,
    conditional_layers_dim=None,  # input or hidden_dim
    # Conv layer overall structure
    depth=4,
    kernel_size=3,
    growth=1.0,
    dilation_growth=2,
    dilation_period=5,
    glu=1,
    conv_dropout=0.2,
    dropout_input=0.1,
    batch_norm=True,
    half=True,
    cnn_pos_encoding=False,
    # Quantizer
    quantizer=False,
    num_codebooks=0,
    codebook_size=0,
    quantizer_commitment=0,
    quantizer_temp_init=0,
    quantizer_temp_min=0,
    quantizer_temp_decay=0,
    # Transformers Encoders
    transformer_input="continuous",
    transformer_encoder_emb="sinusoidal",
    transformer_encoder_layers=4,
    transformer_encoder_heads=4,
    # Conformer encoder variant
    rnn_type="conformer",
    depthwise_conv_kernel_size=15,
    use_group_norm=False,
    convolution_first=False,
    # Transformer Decoders
    transformer_decoder_emb=None,
    transformer_decoder_layers=0,
    transformer_decoder_heads=0,
    transformer_decoder_dim=0,
)

config = TrainingConfigV2(
    brain_encoder_config=model_config,
    data_partition=data_partition,
    # Ada lora settings
    # Around 100k total batches an epoch for gwilliams
    use_adalora=True,
    adalora_init_r=12,
    adalora_target_r=4,
    adalora_tinit=(654 * 3),  # 5% total steps
    adalora_tfinal=(654 * 8),  # 50-80% total steps
    adalora_deltaT=(654 * 1),  # 1-5% total steps
    adalora_lora_alpha=32,
    adalora_lora_dropout=0.1,
    adalora_total_step=(654 * 50),
    # Pre-processing parameters
    # Brain
    new_freq=200,
    frequency_bands={"all": (0.5, 80)},
    max_random_shift=1.0,
    window_size=4,
    window_stride=1,
    brain_clipping=None,
    baseline_window=0.5,
    notch_filter=True,
    scaling="both",
    delay=0.15,
    # Audio
    audio_model="openai/whisper-tiny.en",
    # Hyperparameters
    learning_rate=1e-4,
    weight_decay=1e-4,
    epochs=50,
    steps_per_epoch=450, # 654,
    batch_size=128,
    random_test_size=10,
    seed=42,
    mel_alignment_objectives = {
        "clip_loss": 0.6,
        "mse_loss": 0.4
    },
    latent_alignment_objectives= {
        "cosine_similarity": 0.4,
        "mse_loss": 0.4,
        "clip_loss": 0.6,
        "mmd_loss": 0.0
    },
    decode_timestamps=True,
)

config.brain_encoder_config.mel_normalization = False
config.learning_rate = 3e-4
config.batch_size = 256
config.steps_per_epoch = 654 # 654
config.epochs = 50

config.brain_encoder_config.in_channels = 269
config.decode_timestamps = True

# Sensor layout settings
config.brain_encoder_config.layout_dim=3
config.brain_encoder_config.layout_proj=False
config.brain_encoder_config.layout_scaling="midpoint"
# Merger with spatial attn
config.brain_encoder_config.merger=True
config.brain_encoder_config.merger_emb_type='mlp'
config.brain_encoder_config.merger_emb_dim=256
config.brain_encoder_config.merger_channels=269
config.brain_encoder_config.merger_dropout=0.1  # Float
config.brain_encoder_config.merger_conditional=None

# config.brain_clipping = 20

# config.brain_encoder_config.hidden_dim = 1024
# config.brain_encoder_config.initial_linear = 1024

session = TrainingSessionV2(
    config=config,
    studies={study: "audiotext" for study in data_partition.keys()},
    data_path="data",
    save_path="saves/phase3/combining/channel_merger_269_mlp_256_midpoint_3d",
    clear_cache=False,
    cache_name="/home/ubuntu/cache",
    download_studies=True,
)


# session = load_training_session(
#     save_path="saves/phase3/objectives/baseline_gwilliams_latent_loss_no_latent_alignment/epoch_39",
#     studies={"gwilliams2023": "audiotext"},
#     data_path="data",
#     cache_name="/home/ubuntu/cache",
# )

try:
    session.train(
        device="cuda",
        buffer_size=30,
        num_workers=(multiprocessing.cpu_count() - 2),
        max_cache_size=800,
        current_epoch=0,
    )
except KeyboardInterrupt as e:
    print("Exited")

# try:
#     session.pre_process_all_recordings(
#         buffer_size=30, num_workers=multiprocessing.cpu_count() - 10, max_cache_size=800
#     )
# except KeyboardInterrupt as e:
#     print("Exited")

Loading Gwilliams2023 with batch type audiotext
Loading Armeini2022 with batch type audiotext
Data partitioned on studies ['gwilliams2023', 'armeini2022'].
Train: 159, Unseen Task: 51, Unseen Subject: 12, Unseen Both: 4.

RNNEncoder initialized as conformer with 4 layers, 256 d_model, 4 nhead
	Embedding: sinusoidal, params: 6075392
SimpleConv initialized with 9079914 parameters, cond: ['study', 'subject']
Merger True, merger channels 269
ConvBlocks: 4, hidden_dim: 256, params 2626048
Using torch.bfloat16
Found 64 target modules for AdaLora: ['model.encoder.layers.0.self_attn.k_proj', 'model.encoder.layers.0.self_attn.v_proj', 'model.encoder.layers.0.self_attn.q_proj', 'model.encoder.layers.0.self_attn.out_proj', 'model.encoder.layers.0.fc1', 'model.encoder.layers.0.fc2', 'model.encoder.layers.1.self_attn.k_proj', 'model.encoder.layers.1.self_attn.v_proj', 'model.encoder.layers.1.self_attn.q_proj', 'model.encoder.layers.1.self_attn.out_proj', 'model.encoder.layers.1.fc1', 'model.encoder

Training Epoch 1: 100%|██████████| 159/159 [20:03<00:00,  7.57s/it]


Testing done in 8.21m.
Epoch 1 done in 28.28m. 0.18m/recording.


New best epoch 1 with CER 4.0195 and BLEU 0.0037.
Mel Loss: 6.2299, Clip Loss: 9.5205, MSE: 1.2939
Mel accuracy: 0.0043, Top 5: 0.0226, Top 10: 0.0448


Training Epoch 2: 100%|██████████| 159/159 [19:14<00:00,  7.26s/it]


Testing done in 1.30m.
Epoch 2 done in 20.54m. 0.13m/recording.


New best epoch 2 with CER 0.0000 and BLEU 0.0000.
Mel Loss: 5.8909, Clip Loss: 9.5206, MSE: 0.4463
Mel accuracy: 0.0046, Top 5: 0.0226, Top 10: 0.0448


...

Testing done in 4.10m.
Epoch 21 done in 23.71m. 0.15m/recording.


Training Epoch 22: 100%|██████████| 159/159 [19:36<00:00,  7.40s/it]


Testing done in 4.21m.
Epoch 22 done in 23.82m. 0.15m/recording.


Training Epoch 23: 100%|██████████| 159/159 [19:38<00:00,  7.41s/it]


Testing done in 4.25m.
Epoch 23 done in 23.89m. 0.15m/recording.
Early stopping at epoch 23. Highest metrics at epoch 12.


Training completed. Highest epoch at 12.


Test unseen_subject at epoch 12. Mel Loss: 5.8548, Clip Loss: 9.4214, MSE: 0.5047
Mel accuracy: 0.0463, Top 5: 0.1432, Top 10: 0.2430
BLEU: 0.0069, ROUGE-1: 0.0478, BERT: 0.3626, CER: 0.9433, SELF-BLEU: 0.5101


Test unseen_task at epoch 12. Mel Loss: 5.8786, Clip Loss: 9.4680, MSE: 0.4946
Mel accuracy: 0.0431, Top 5: 0.1208, Top 10: 0.1947
BLEU: 0.0034, ROUGE-1: 0.0260, BERT: 0.3591, CER: 0.9345, SELF-BLEU: 0.4907


Test unseen_both at epoch 12. Mel Loss: 5.8590, Clip Loss: 9.4356, MSE: 0.4940
Mel accuracy: 0.0505, Top 5: 0.1256, Top 10: 0.2488
BLEU: 0.0033, ROUGE-1: 0.0220, BERT: 0.3571, CER: 0.9256, SELF-BLEU: 0.5224
